In [3]:
!pip install pmdarima
!pip install prophet

In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error

import pmdarima
from pmdarima.utils import plot_acf, plot_pacf
from pmdarima.preprocessing import BoxCoxEndogTransformer
from pmdarima.arima import auto_arima, ADFTest
from pmdarima.metrics import smape

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing

import lightgbm

import prophet

import tensorflow as tf
import tensorflow.keras as keras

from pylab import rcParams
rcParams['figure.figsize'] = (12, 6)
rcParams['xtick.labelsize'] = 12
rcParams['ytick.labelsize'] = 12
rcParams['axes.labelsize'] = 12

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 80 from PyObject

In [ ]:
train_df = pd.read_csv('DailyDelhiClimateTrain.csv')

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
print(f"Min date: {train_df.date.min()}\nMax date: {train_df.date.max()}") 

In [ ]:
train_df.index = pd.DatetimeIndex(pd.to_datetime(train_df['date']), freq='D')
train_df.drop('date', axis=1, inplace=True)

In [ ]:
train_df.head()

## Checking the data distributions of the independent features

### Mean Pressure

In [ ]:
sns.histplot(train_df['meanpressure'],)
plt.show()

In [ ]:
# https://www.theweatherprediction.com/habyhints3/909/
# https://barometricpressure.app/new-delhi
f = (train_df['meanpressure'] < 1050) & (train_df['meanpressure'] > 950)
sns.histplot(
    train_df[f]['meanpressure'],
)
plt.show()

In [ ]:
train_df[train_df['meanpressure'] < 950]

In [ ]:
train_df[train_df['meanpressure'] > 1050]

In [ ]:
sns.lineplot(
    x=train_df[f].index,
    y=train_df[f]['meanpressure']
)
plt.show()

In [ ]:
#TODO create function to impute pressure of day before for outliers

### Wind Speed

In [ ]:
sns.histplot(train_df['wind_speed'],)
plt.show()

In [ ]:
def detect_outliers_iqr(data):
  p25 = np.percentile(data, 25)
  p75 = np.percentile(data, 75)
  iqr = p75 - p25
  lower_bound, upper_bound = (p25 - 1.5 * iqr, p75 + 1.5 * iqr)
  f = (data < lower_bound) | (data > upper_bound)
  return data[f], data[~f] 

In [ ]:
outliers, cleaned_series = detect_outliers_iqr(train_df['wind_speed'])
train_df.loc[outliers.index]

In [ ]:
sns.lineplot(
    x=train_df.index,
    y=train_df['wind_speed']
)
plt.show()

In [ ]:
sns.lineplot(
    x=train_df[train_df['wind_speed'] < 20].index,
    y=train_df[train_df['wind_speed'] < 20]['wind_speed']
)
plt.show()

In [ ]:
#TODO impute the wind_speed to the one from the day before

### Humidity

In [ ]:
sns.histplot(train_df['humidity'],)
plt.show()

In [ ]:
sns.lineplot(
    x=train_df.index,
    y=train_df['humidity']
)
plt.show()

## Description of the target variable

In [ ]:
sns.histplot(train_df['meantemp'],)
plt.show()

In [ ]:
sns.lineplot(
    x=train_df.index,
    y=train_df['humidity']
)
plt.show()

In [ ]:
plot_acf(train_df['humidity'])

In [ ]:
plot_pacf(train_df['humidity'])

In [ ]:
result = seasonal_decompose(train_df['meantemp'])
_ = result.plot()

In [ ]:
adf_test = ADFTest()
adf_test.should_diff(train_df['meantemp'])

In [ ]:
boxcox_transfomer = BoxCoxEndogTransformer()
boxcox_series = boxcox_transfomer.fit_transform(train_df['meantemp'])

In [ ]:
boxcox_series[0].shape

In [ ]:
train_df

## Modelling

In [ ]:
val_periods = 180
training_df = train_df.iloc[:train_df.shape[0]-val_periods, :]
val_df = train_df.iloc[-val_periods:, :]

In [ ]:
def evaluate_model(y_true, y_hat):
  pred_smape = smape(y_true, y_hat)
  pred_mae = mean_absolute_error(y_true, y_hat)
  pred_mse = mean_squared_error(y_true, y_hat)
  print(f"SMAPE: {pred_smape}\nMAE: {pred_mae}\nMSE: {pred_mse}")
  return pred_smape, pred_mae, pred_mse

### ARIMA

In [ ]:
arima_model = auto_arima(
    training_df['meantemp'], start_d=1, max_d=12, max_q=12,
    seasonal=True, m=12, max_D=12, max_Q=12, stationary=False,
    n_jobs=-1, verbose=True
)

In [ ]:
print(arima_model.summary())

In [ ]:
arima_model.plot_diagnostics()
plt.show()

In [ ]:
arima_preds = arima_model.predict(val_periods)
arima_smape, arima_mae, arima_mse = evaluate_model(val_df['meantemp'], arima_preds)

In [ ]:
plt.plot(val_df['meantemp'], color='blue', label='Real Values')
plt.plot(val_df.index, arima_preds, color='red', label='Predicted Values')
plt.legend()
plt.show()

### Holt-Winters

In [ ]:
hw_trend = 'additive'
hw_seasonal = 'additive'

hw_model = ExponentialSmoothing(
    training_df['meantemp'], trend=hw_trend, seasonal=hw_seasonal,
    seasonal_periods=12,
)
hw_model = hw_model.fit()
hw_preds = hw_model.forecast(val_periods)

In [ ]:
hw_smape, hw_mae, hw_mse = evaluate_model(val_df['meantemp'], hw_preds)

In [ ]:
plt.plot(val_df['meantemp'], color='blue', label='Real Values')
plt.plot(hw_preds, color='red', label='Predicted Values')
plt.legend()
plt.show()

### Prophet

In [ ]:
m = prophet.Prophet()
m.fit(training_df['meantemp'].reset_index().rename({'date': 'ds', 'meantemp': 'y'}, axis=1))

In [ ]:
future = m.make_future_dataframe(periods=val_periods, include_history=True)
prophet_preds = m.predict(future)['yhat']

In [ ]:
prophet_smape, prophet_mae, prophet_mse = evaluate_model(val_df['meantemp'], prophet_preds.iloc[-val_periods:])

In [ ]:
plt.plot(val_df['meantemp'], color='blue', label='Real Values')
plt.plot(val_df.index, prophet_preds.iloc[-val_periods:], color='red', label='Predicted Values')
plt.legend()
plt.show()

In [ ]:
plt.plot(training_df['meantemp'], color='green', label='Train Values')
plt.plot(val_df['meantemp'], color='blue', label='Validation Values')
plt.plot(pd.date_range(start=training_df.index.min(), end=val_df.index.max()), prophet_preds, color='red', label='Predicted Values')
plt.legend()
plt.show()

## Linear Regression

In [ ]:
def extract_date_features(original_df):
  df = original_df.copy()
  df['year'] = df.index.year
  df['yearday'] = df.index.day_of_year
  df['yearday_sin'] = np.sin(2 * np.pi * df['yearday']/df['yearday'].max())
  df['yearday_cos'] = np.cos(2 * np.pi * df['yearday']/df['yearday'].max())
  df['month'] = df.index.month
  df['monthday'] = df.index.day
  df['monthday_sin'] = np.sin(2 * np.pi * df['monthday']/df['monthday'].max())
  df['monthday_cos'] = np.cos(2 * np.pi * df['monthday']/df['monthday'].max())
  df['weekday'] = df.index.weekday
  df['weekday_sin'] = np.sin(2 * np.pi * df['weekday']/df['weekday'].max())
  df['weekday_cos'] = np.cos(2 * np.pi * df['weekday']/df['weekday'].max())
  return df

In [ ]:
training_df_featurized = extract_date_features(training_df)

In [ ]:
val_df_featurized = extract_date_features(val_df)

In [ ]:
X = training_df_featurized[[
  'yearday_sin', 'yearday_cos', 'monthday_sin', 'monthday_cos', 'weekday_sin', 'weekday_cos',
]]
y = training_df_featurized.meantemp

In [ ]:
linear_model = LinearRegression()
linear_model.fit(X, y)
linear_preds = linear_model.predict(
  val_df_featurized[[
  'yearday_sin', 'yearday_cos', 'monthday_sin', 'monthday_cos', 'weekday_sin', 'weekday_cos',
]]
)

In [ ]:
linear_smape, linear_mae, linear_mse = evaluate_model(val_df['meantemp'], linear_preds)

In [ ]:
plt.plot(val_df['meantemp'], color='blue', label='Real Values')
plt.plot(val_df.index, linear_preds, color='red', label='Predicted Values')
plt.legend()
plt.show()

### LightGBM

In [ ]:
light_model = lightgbm.LGBMRegressor()
light_model.fit(X, y)
light_preds = light_model.predict(
  val_df_featurized[[
  'yearday_sin', 'yearday_cos', 'monthday_sin', 'monthday_cos', 'weekday_sin', 'weekday_cos',
]]
)

In [ ]:
light_smape, light_mae, light_mse = evaluate_model(val_df['meantemp'], light_preds)

In [ ]:
plt.plot(val_df['meantemp'], color='blue', label='Real Values')
plt.plot(val_df.index, light_preds, color='red', label='Predicted Values')
plt.legend()
plt.show()

### Univariate Recurrent Neural Network

In [ ]:
#TODO: Scale input and/or trend

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):  
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.shuffle(shuffle_buffer)
    
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

In [ ]:
def plot_lr_curves(model, optimizer, criterion, start_lr=1e-6, epochs=50):
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(
        lambda epoch: start_lr * 10**(epoch / 10))
    model.compile(loss=criterion, optimizer=optimizer)
    history = model.fit(train_set, epochs=epochs, callbacks=[lr_schedule])

    lrs = start_lr * (10 ** (np.arange(epochs) / 10))
    plt.grid(True)
    plt.semilogx(lrs, history.history["loss"])
    plt.tick_params('both', length=10, width=1, which='both')
    plt.show()

In [ ]:
def model_forecast(model, series, window_size, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)

    dataset = dataset.flat_map(lambda w: w.batch(window_size))    
    dataset = dataset.batch(batch_size).prefetch(1)
    
    forecast = model.predict(dataset)
    
    return forecast

In [ ]:
window_size = 30
batch_size = 64
shuffle_buffer_size = 1000

hidden_dim_rnn = 64
hidden_dim_fc = 32
dropout = 0.4

criterion = keras.losses.Huber()
optimizer = keras.optimizers.Adam(3e-4)
epochs = 100

In [ ]:
train_set = windowed_dataset(training_df['meantemp'], window_size, batch_size, shuffle_buffer_size)
val_set = windowed_dataset(val_df['meantemp'], window_size, batch_size, shuffle_buffer_size)

In [ ]:
model = tf.keras.models.Sequential([
  keras.layers.Conv1D(filters=64, kernel_size=3,
                      strides=1,
                      activation=tf.nn.relu,
                      padding='causal',
                      input_shape=[window_size, 1]),
  keras.layers.Dropout(dropout),
  keras.layers.LSTM(hidden_dim_rnn, return_sequences=True, dropout=dropout, recurrent_dropout=dropout),
  keras.layers.LSTM(hidden_dim_rnn, dropout=dropout, recurrent_dropout=dropout),
  keras.layers.Dense(hidden_dim_fc, activation=tf.nn.relu),
  keras.layers.Dropout(dropout),
  keras.layers.Dense(hidden_dim_fc, activation=tf.nn.relu),
  keras.layers.Dropout(dropout),
  keras.layers.Dense(1),
])

init_weights = model.get_weights()
model.summary()

In [ ]:
plot_lr_curves(model, keras.optimizers.Adam(), criterion)

In [ ]:
model.compile(loss=criterion, optimizer=optimizer, metrics=['mae', 'mse'])

In [ ]:
model.set_weights(init_weights)
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr_on_plateau = keras.callbacks.ReduceLROnPlateau(patience=5,)
history = model.fit(train_set, validation_data=val_set, epochs=epochs, callbacks=[early_stopping, reduce_lr_on_plateau])

In [ ]:
plt.plot(history.history['loss'], color='red', label='Training Loss')
plt.plot(history.history['val_loss'], color='blue', label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['mae'], color='red', label='Training MAE')
plt.plot(history.history['val_mae'], color='blue', label='Validation MAE')
plt.legend()
plt.show()

In [ ]:
forecast_series = train_df.iloc[val_periods-window_size:-1, :]['meantemp']
rnn_preds = model_forecast(model, forecast_series, window_size, batch_size).squeeze()

In [ ]:
rnn_smape, rnn_mae, rnn_mse = evaluate_model(val_df['meantemp'], rnn_preds[-val_periods:])

In [ ]:
plt.plot(val_df['meantemp'], color='blue', label='Real Values')
plt.plot(val_df.index, rnn_preds[-val_periods:], color='red', label='Predicted Values')
plt.legend()
plt.show()